In [1]:
import numpy as np
import astropy.units as u
import os
from astropy.constants import M_sun
from astropy.coordinates import Distance
import matplotlib.pyplot as plt

from agnpy.targets import SphericalShellBLR, lines_dictionary

from agnpy.absorption import Absorption
from agnpy.utils.plot import load_mpl_rc

from gammapy.modeling.models import (
    EBLAbsorptionNormSpectralModel,
    LogParabolaSpectralModel
)

from gammapy.modeling import Parameter

import agnpy 

from astropy.io import fits
from astropy import units as u

from scipy import integrate

from scipy.integrate import quad, nquad


from agnpy.utils.math import (
    axes_reshaper,
    log,
    mu_to_integrate,
    phi_to_integrate,
    min_rel_distance
)

from agnpy.utils.geometry import (
    cos_psi,
    x_re_shell,
    mu_star_shell,
    x_re_ring,
    x_re_ring_mu_s,
    phi_mu_re_shell,
    phi_mu_re_ring,
    x_re_shell_mu_s,
)

from agnpy.utils.conversion import nu_to_epsilon_prime

from astropy.constants import c, G, m_e, sigma_T

dobrze byłby jakbyś przedyskutował z Cosimo tę implementacje absorpcji po liście linii, bo to ciągle jest tylko w Twoim prywatnym kodzie i to czy można przyspieszyć obliczenia (i zrobić je bardziej odpornymi na neiwielkie zmiany parametrów wejściowych)
https://docs.scipy.org/doc/scipy/tutorial/integrate.html tutaj jest kilka metod do całek
w szczególności ta: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.nquad.html#scipy.integrate.nquad

In [2]:
import scipy.integrate

In [9]:
func = lambda x0,x1,x2,x3 : x0**2 + x1*x2 - x3**3 + np.sin(x0) + (
                                1 if (x0-.2*x3-.5-.25*x1>0) else 0)

def opts0(*args, **kwargs):
    return {'points':[0.2*args[2] + 0.5 + 0.25*args[0]]}

In [10]:
integrate.nquad(func, [[0,1], [-1,1], [.13,.8], [-.15,1]],
                opts=[opts0,{},{},{}], full_output=True)

(1.5267454070738635, 2.943736000140233e-14, {'neval': 388962})

In [11]:
integrate.nquad(func, [[0,1], [-1,1], [.13,.8], [-.15,1]], opts=[opts0,{},{},{}])

(1.5267454070738635, 2.943736000140233e-14)

In [42]:
f = lambda x,a : a*x

y, err = integrate.quad(f, 0, 1, args=(1,))

In [37]:
opts0()

IndexError: tuple index out of range

In [4]:
def test_matching_quad(self):
    def func(x):
        return x**2 + 1

    res, reserr = quad(func, 0, 4)
    res2, reserr2 = nquad(func, ranges=[[0, 4]])
    assert_almost_equal(res, res2)
    assert_almost_equal(reserr, reserr2)

In [5]:
def func(x):
    return x**2 + 1

In [6]:
res, reserr = quad(func, 0, 5)
res2, reserr2 = nquad(func, ranges=[[0, 5]])

In [27]:
N = 5

def f(t, x):
    return np.exp(-x*t) / t**N



In [30]:
integrate.nquad(f, [[1, np.inf],[0, np.inf]])

(0.2000000000189363, 1.3682975855986131e-08)

In [35]:
integrate.nquad(f, [[1, np.inf],[0, np.inf]], opts={'epsabs':0.01, 'epsrel':0.01})

(0.19999889370959617, 0.0024561464081052395)

In [10]:
integrate.nquad?

In [19]:
res2

46.66666666666667

In [21]:
np.trapz?

In [28]:
E = np.logspace(2, 6) * u.MeV
nu = E.to("Hz", equivalencies=u.spectral())
L_Hbeta=8.93e43*u.Unit("erg s-1")
R_Hbeta=5.93e17 * u.cm
z = 1.032 
xi_line = 0.1    

blr = SphericalShellBLR(L_Hbeta, xi_line, 'Hbeta', R_Hbeta)

In [29]:
blr

In [27]:
for line in list(lines_dictionary.keys()):
    print(line)
    xi_line_this=lines_dictionary[line]['L_Hbeta_ratio']
    R_line_this=R_Hbeta*lines_dictionary[line]['R_Hbeta_ratio']
        
    #blrs[line] = SphericalShellBLR(LHbeta, xi_line_this, line, R_line_this)

Lyepsilon
Lydelta
CIII
NIII
Lybeta
OVI
ArI
Lyalpha
OI
SiII
SiIV
OIV]
CIV
NIV
AlII
CIII]
[NeIV]
MgII
Hdelta
Hgamma
HeII
Hbeta
[ClIII]
HeI
Halpha


In [43]:
def evaluate_tau_blr(
        nu,
        z,
        mu_s,
        L_disk,
        xi_line,
        epsilon_line,
        R_line,
        r,
        l_size=50,
        mu=mu_to_integrate,
        phi=phi_to_integrate,
    ):
        """Evaluates the gamma-gamma absorption produced by a spherical shell
        BLR for a general set of model parameters

        Parameters
        ----------
        nu : :class:`~astropy.units.Quantity`
            array of frequencies, in Hz, to compute the tau
            **note** these are observed frequencies (observer frame)
        z : float
            redshift of the source
        mu_s : float
            cosine of the angle between the blob motion and the jet axis
        L_disk : :class:`~astropy.units.Quantity`
            Luminosity of the disk whose radiation is being reprocessed by the BLR
        xi_line : float
            fraction of the disk radiation reprocessed by the BLR
        epsilon_line : string
            dimensionless energy of the emitted line
        R_line : :class:`~astropy.units.Quantity`
            radius of the BLR spherical shell
        r : :class:`~astropy.units.Quantity`
            distance between the Broad Line Region and the blob
        l_size : int
            size of the array of distances from the BH to integrate over
        mu, phi : :class:`~numpy.ndarray`
            arrays of cosine of zenith and azimuth angles to integrate over

        Returns
        -------
        :class:`~astropy.units.Quantity`
            array of the tau values corresponding to each frequency
        """
        # conversions
        epsilon_1 = nu_to_epsilon_prime(nu, z)
        # multidimensional integration
        l = np.logspace(0, 5, l_size) * r

        # check if any point is too close to R_line, the function works only for mu=1, so
        # we can check directly if R_line is within 'l' array
        idx = np.isclose(l, R_line, rtol=min_rel_distance)
        l[idx] += min_rel_distance * R_line

        _mu, _phi, _l, _epsilon_1 = axes_reshaper(mu, phi, l, epsilon_1)
        x = x_re_shell(_mu, R_line, _l)
        _mu_star = mu_star_shell(_mu, R_line, _l)
        _cos_psi = cos_psi(mu_s, _mu_star, _phi)
        s = _epsilon_1 * epsilon_line * (1 - _cos_psi) / 2
        integrand = (1 - _cos_psi) / x ** 2 * sigma(s)
        # integrate
        integral_mu = np.trapz(integrand, mu, axis=0)
        integral_phi = np.trapz(integral_mu, phi, axis=0)
        integral = np.trapz(integral_phi, l, axis=0)
        print(integral)
        prefactor = (L_disk * xi_line) / (
            (4 * np.pi) ** 2 * epsilon_line * m_e * c ** 3
        )
        return (prefactor * integral).to_value("")
    
def sigma(s):
    """photon-photon pair production cross section, Eq. 17 of [Dermer2009]"""
    beta_cm = np.sqrt(1 - 1 / s)
    prefactor = 3 / 16 * sigma_T * (1 - np.power(beta_cm, 2))
    term1 = (3 - np.power(beta_cm, 4)) * log((1 + beta_cm) / (1 - beta_cm))
    term2 = -2 * beta_cm * (2 - np.power(beta_cm, 2))
    values = prefactor * (term1 + term2)
    values[s < 1] = 0
    return values

In [44]:

E = np.logspace(2, 6) * u.MeV
nu = E.to("Hz", equivalencies=u.spectral())
nu = 3*1e25*u.Hz
z = 1.032 
mu_s = 1
L_disk=8.93e43*u.Unit("erg s-1")
R_line=5.93e17 * u.cm
xi_line = 0.1    
epsilon_line = 4.9896564007565614e-06
r = 1e16*u.cm

result = evaluate_tau_blr(
            nu,
            z,
            mu_s,
            L_disk,
            xi_line,
            epsilon_line,
            R_line,
            r,
            l_size=50,
            mu=mu_to_integrate,
            phi=phi_to_integrate,
        )

[3.9627031e-46] m2 / cm


In [45]:
result

array([1.82979114])

In [8]:
nu

<Quantity 3.e+25 Hz>

$\tau_{\gamma \gamma}(\nu) = \int_{r}^{\infty} {\rm d}l \; \int_{0}^{2\pi} {\rm d}\phi \;
                            \int_{-1}^{1} {\rm d}\mu \; (1 - \cos\psi) \int_{0}^{\infty} {\rm d}\epsilon \;
                            \frac{u(\epsilon, \mu, \phi; l)}{\epsilon m_e c^2} \, \sigma_{\gamma \gamma}(s), $
                            
where: 
  
  - $ \cos\psi = \mu\mu_s + \sqrt{1 - \mu^2}\sqrt{1 - \mu_s^2} \cos\phi$ is the cosine of the angle between the hitting and the absorbing photon;
  
  - $ u(\epsilon, \mu, \phi; l)`$ is the energy density of the target photon field with $\epsilon$ dimensionless energy, $(\mu, \phi)$ solid angle, $l$ distance of the blob from the photon field;
  
  - $ \sigma_{\gamma \gamma}(s)$ is the pair-production cross section, with $s = \epsilon_1 \epsilon \, (1 - \cos\psi)\,/\,2$ and $\epsilon_1 = h \nu\,/\,(m_e c^2)$ the dimensionless hitting photon energy.
  
Basic formulas are borrowed from [Finke2016]_. The approach presented therein (and in [Dermer2009]_) simplifies the integration by assuming that the hitting photons travels in the direction parallel to the jet axis ($\mu_s \rightarrow 1$), decoupling the cross section and the $(1 - \cos\psi)$ term from the integral on $\phi$. The optical depths thus calculated are therefore valid only for blazars.

In [51]:
def cos_psi_test(mu, phi):
    """compute the angle between the blob (with zenith mu_s) and a photon with
    zenith and azimuth (mu, phi). The system is symmetric in azimuth for the
    electron phi_s = 0, Eq. 8 in [Finke2016]_."""
    mu_s = 1
    term_1 = mu * mu_s
    term_2 = np.sqrt(1 - np.power(mu, 2)) * np.sqrt(1 - np.power(mu_s, 2))
    term_3 = np.cos(phi)
    return term_1 + term_2 * term_3

def evaluate_tau_blr(
        nu,
        z,
        mu_s,
        L_disk,
        xi_line,
        epsilon_line,
        R_line,
        r,
        l_size=50,
        mu=mu_to_integrate,
        phi=phi_to_integrate,
    ):
        """Evaluates the gamma-gamma absorption produced by a spherical shell
        BLR for a general set of model parameters

        Parameters
        ----------
        nu : :class:`~astropy.units.Quantity`
            array of frequencies, in Hz, to compute the tau
            **note** these are observed frequencies (observer frame)
        z : float
            redshift of the source
        mu_s : float
            cosine of the angle between the blob motion and the jet axis
        L_disk : :class:`~astropy.units.Quantity`
            Luminosity of the disk whose radiation is being reprocessed by the BLR
        xi_line : float
            fraction of the disk radiation reprocessed by the BLR
        epsilon_line : string
            dimensionless energy of the emitted line
        R_line : :class:`~astropy.units.Quantity`
            radius of the BLR spherical shell
        r : :class:`~astropy.units.Quantity`
            distance between the Broad Line Region and the blob
        l_size : int
            size of the array of distances from the BH to integrate over
        mu, phi : :class:`~numpy.ndarray`
            arrays of cosine of zenith and azimuth angles to integrate over

        Returns
        -------
        :class:`~astropy.units.Quantity`
            array of the tau values corresponding to each frequency
        """
        # conversions
        epsilon_1 = nu_to_epsilon_prime(nu, z)
        # multidimensional integration
        #l = np.logspace(0, 5, l_size) * r

        # check if any point is too close to R_line, the function works only for mu=1, so
        # we can check directly if R_line is within 'l' array
        #idx = np.isclose(l, R_line, rtol=min_rel_distance)
        #l[idx] += min_rel_distance * R_line

        #_mu, _phi, _l, _epsilon_1 = axes_reshaper(mu, phi, l, epsilon_1)
        #x = x_re_shell(_mu, R_line, _l)
        #_mu_star = mu_star_shell(_mu, R_line, _l)
 #       _cos_psi = cos_psi(mu_s, _mu_star, _phi)
        #_cos_psi = cos_psi_test(_mu_star, _phi)
#        s = _epsilon_1 * epsilon_line * (1 - _cos_psi) / 2
        
#        integrand = (1 - _cos_psi) / x ** 2 * sigma(s)
        
        def f(mu, phi, log_l):
            l = np.exp(log_l)
            mu_star = mu_star_shell(mu, R_line.to_value('cm'), l)
            _cos_psi = cos_psi(mu_s, mu_star, phi)
            x = x_re_shell(mu, R_line.to_value('cm'), l)
            s = epsilon_1 * epsilon_line * (1 - _cos_psi) / 2
            integrand = (1 - _cos_psi)*l / x ** 2 * sigma(s).to_value("cm**2")
            return integrand
        
        I = integrate.nquad(f, ranges=[[-1, 1], [0, 2*np.pi], [np.log(r.to_value('cm')), np.log(1e5*r.to_value('cm'))]], 
                           opts={'epsabs':0.01, 'epsrel':0.01})
        print(I)
        integral = I*u.cm
    
        #I = 2*np.pi*r*2/ (x**2 * sigma(s))
        #print(I)
        
#        integral_mu = np.trapz(integrand, mu, axis=0)
#        integral_phi = np.trapz(integral_mu, phi, axis=0)
#        integral = np.trapz(integral_phi, l, axis=0)

        #integral_mu = scipy.integrate.simps(integrand, mu, axis=0)
        #integral_phi = scipy.integrate.simps(integral_mu, phi, axis=0)
        #I = scipy.integrate.simps(integral_phi, l, axis=0)
        #integral = I*u.m**2 / u.cm
        #print(integral)
            
        prefactor = (L_disk * xi_line) / (
            (4 * np.pi) ** 2 * epsilon_line * m_e * c ** 3
        )
        return (prefactor * integral).to_value("")
        
def fun1(mu, phi, l):
    
    return (1 - mu)
    
def sigma(s):
    """photon-photon pair production cross section, Eq. 17 of [Dermer2009]"""
    beta_cm = np.sqrt(1 - 1 / s)
    prefactor = 3 / 16 * sigma_T * (1 - np.power(beta_cm, 2))
    term1 = (3 - np.power(beta_cm, 4)) * log((1 + beta_cm) / (1 - beta_cm))
    term2 = -2 * beta_cm * (2 - np.power(beta_cm, 2))
    values = prefactor * (term1 + term2)
    values[s < 1] = 0
    return values

In [52]:

E = np.logspace(2, 6) * u.MeV
nu = E.to("Hz", equivalencies=u.spectral())
nu = 3*1e25*u.Hz
z = 1.032 
mu_s = 1
L_disk=8.93e43*u.Unit("erg s-1")
R_line=5.93e17 * u.cm
xi_line = 0.1    
epsilon_line = 4.9896564007565614e-06
r = 1e16*u.cm

evaluate_tau_blr(
            nu,
            z,
            mu_s,
            L_disk,
            xi_line,
            epsilon_line,
            R_line,
            r,
            l_size=50,
            mu=mu_to_integrate,
            phi=phi_to_integrate,
        )

(2.5226906538002067e-42, 2.7933023486441275e-42)


array([1.16486068, 1.28981651])

In [48]:
np.log(2.71)

0.9969486348916096

In [41]:
prefactor = (L_disk * xi_line) / (
            (4 * np.pi) ** 2 * epsilon_line * m_e * c ** 3
        )
integ = 2.5079860552646397e-42*u.cm
my_result = (prefactor * integ).to_value("")

In [42]:
my_result

1.1580707798251888

In [18]:
r.to_value('cm')

1e+16

In [77]:
I = evaluate_tau_blr(
            nu,
            z,
            mu_s,
            L_disk,
            xi_line,
            epsilon_line,
            R_line,
            r,
            l_size=50,
            mu=mu_to_integrate,
            phi=phi_to_integrate,
        )
print(I)

[3.9627031e-46] m2 / cm
[1.82979114]


In [92]:


I = evaluate_tau_blr(
            nu,
            z,
            mu_s,
            L_disk,
            xi_line,
            epsilon_line,
            R_line,
            r,
            l_size=50,
            mu=mu_to_integrate,
            phi=phi_to_integrate,
        )
print(I)

[3.76046545e-46] m2 / cm
[1.73640724]


In [86]:
2**7

128

In [23]:
integrand.shape

(100, 50, 50, 1)

In [24]:
mu.shape

(100,)

In [25]:
integral_mu.shape

(50, 50, 1)

In [26]:
mu

array([-1.        , -0.97979798, -0.95959596, -0.93939394, -0.91919192,
       -0.8989899 , -0.87878788, -0.85858586, -0.83838384, -0.81818182,
       -0.7979798 , -0.77777778, -0.75757576, -0.73737374, -0.71717172,
       -0.6969697 , -0.67676768, -0.65656566, -0.63636364, -0.61616162,
       -0.5959596 , -0.57575758, -0.55555556, -0.53535354, -0.51515152,
       -0.49494949, -0.47474747, -0.45454545, -0.43434343, -0.41414141,
       -0.39393939, -0.37373737, -0.35353535, -0.33333333, -0.31313131,
       -0.29292929, -0.27272727, -0.25252525, -0.23232323, -0.21212121,
       -0.19191919, -0.17171717, -0.15151515, -0.13131313, -0.11111111,
       -0.09090909, -0.07070707, -0.05050505, -0.03030303, -0.01010101,
        0.01010101,  0.03030303,  0.05050505,  0.07070707,  0.09090909,
        0.11111111,  0.13131313,  0.15151515,  0.17171717,  0.19191919,
        0.21212121,  0.23232323,  0.25252525,  0.27272727,  0.29292929,
        0.31313131,  0.33333333,  0.35353535,  0.37373737,  0.39

In [13]:
evaluate_tau_blr(
            nu,
            z,
            mu_s,
            L_disk,
            xi_line,
            epsilon_line,
            R_line,
            r,
            l_size=50,
            mu=mu_to_integrate,
            phi=phi_to_integrate,
        )

[[[[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]

  ...

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]]


 [[[2.40559616e-02]
   [2.38479810e-02]
   [2.35887195e-02]
   ...
   [2.21137568e-08]
   [1.38276962e-08]
   [8.64573216e-09]]

  [[2.40559616e-02]
   [2.38479810e-02]
   [2.35887195e-02]
   ...
   [2.21137568e-08]
 

/home/pawel1/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/pawel1/anaconda3/envs/gammapy-0.18.2/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [5]:
integrand.shape

(100, 50, 50, 1)

In [6]:
mu

array([-1.        , -0.97979798, -0.95959596, -0.93939394, -0.91919192,
       -0.8989899 , -0.87878788, -0.85858586, -0.83838384, -0.81818182,
       -0.7979798 , -0.77777778, -0.75757576, -0.73737374, -0.71717172,
       -0.6969697 , -0.67676768, -0.65656566, -0.63636364, -0.61616162,
       -0.5959596 , -0.57575758, -0.55555556, -0.53535354, -0.51515152,
       -0.49494949, -0.47474747, -0.45454545, -0.43434343, -0.41414141,
       -0.39393939, -0.37373737, -0.35353535, -0.33333333, -0.31313131,
       -0.29292929, -0.27272727, -0.25252525, -0.23232323, -0.21212121,
       -0.19191919, -0.17171717, -0.15151515, -0.13131313, -0.11111111,
       -0.09090909, -0.07070707, -0.05050505, -0.03030303, -0.01010101,
        0.01010101,  0.03030303,  0.05050505,  0.07070707,  0.09090909,
        0.11111111,  0.13131313,  0.15151515,  0.17171717,  0.19191919,
        0.21212121,  0.23232323,  0.25252525,  0.27272727,  0.29292929,
        0.31313131,  0.33333333,  0.35353535,  0.37373737,  0.39

In [32]:
res2, reserr2 = nquad(sigma, [[1.1,2]])

In [33]:
res2

1.3449059869058106e-29

In [25]:
sigma_T.value

6.6524587321e-29